This script is used to find out how model performance changes with number of predictors for the full-predictor, non-first-term model.

In [2]:
import pickle
import pandas as pd
import numpy as np
from collections import Counter
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.stats.mstats import gmean
import seaborn as sns
import matplotlib.pyplot as plt

results_dir = "~\\Box Sync\\Clickstream\\evaluation_results\\full\\updated\\"

In [3]:
sn_dict = {"Blue Ridge": "BRCC",
           "Central Virginia": "CVCC",
           "Dabney S. Lancaster": "DSLCC",
           "Danville": "DCC",
           "Eastern Shore": "ESCC",
           "Germanna": "GCC",
           'J. Sargeant Reynolds': "JSRCC",
           'John Tyler': "JTCC",
           "Lord Fairfax": "LFCC",
           "Mountain Empire": "MECC",
           "New River": "NRCC",
           "Northern Virginia": "NVCC",
           "Patrick Henry": "PHCC",
           "Paul D. Camp": "PDCCC",
           "Piedmont Virginia": "PVCC",
           "Rappahannock": "RCC",
           "Southside Virginia": "SSVCC",
           "Southwest Virginia": "SWVCC",
           "Thomas Nelson": "TNCC",
           "Tidewater": "TCC",
           "Virginia Highlands": "VHCC",
           "Virginia Western": "VWCC",
           "Wytheville": "WCC"}

In [4]:
df0 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\full\\updated\\LMS_data_final_full_new.csv")
df1 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\full\\updated\\course_specific_predictors_new.csv")
df2 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\full\\updated\\term_specific_predictors_new.csv")
for v in [int(e) for e in np.unique(df2.cip) if e != 0]:
    df2.loc[:,'cip_'+str(v)] = (df2.cip == v).astype(int)
for v in [int(e) for e in np.unique(df2.degree_level) if e != 4]:
    df2.loc[:,'degree_level_'+str(v)] = (df2.degree_level == v).astype(int)
df2 = df2.drop(['cip', 'degree_level'], axis=1)
df3 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\full\\updated\\cluster_specific_predictors.csv")
df4 = pd.read_stata("~\\Box Sync\\Clickstream\\data\\full\\updated\\instructor_related_predictors.dta")
df5 = df0.loc[:,['vccsid','strm','college','course','section']].copy()
df5.loc[:,'college_new'] = df5.college.apply(lambda x: sn_dict[x])
for sn in [e for e in sn_dict.values() if e != "BRCC"]:
    df5.loc[:,'college_'+sn] = (df5.college_new == sn).astype(int)
df5 = df5.drop(['college_new'], axis=1)
df = df0.merge(df1, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df2, how='inner', on=['vccsid','strm'])\
.merge(df3, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df4, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df5, how='inner', on=['vccsid','strm','college','course','section'])
predictors = [e for e in list(df.columns)[5:] if e != "grade"]
len(predictors)

329

In [5]:
assert pd.isnull(df).any().any() == False

In [6]:
df.shape

(969025, 335)

In [7]:
train_df = df[df.strm != 2212]
test_df = df[df.strm == 2212]
original_test_grade = np.array(test_df.grade)
train_df.loc[:,'grade'] = train_df.apply(lambda x: 1 if x.loc['grade'] in {'A','B','C'} else 0, axis=1)
test_df.loc[:,'grade'] = test_df.apply(lambda x: 1 if x.loc['grade'] in {'A','B','C'} else 0, axis=1)
print(train_df.shape,test_df.shape)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(698361, 335) (270664, 335)


In [8]:
# raw_predictors = {\
# 10: 'pct_withdrawn, cum_gpa, tot_click_cnt_qrt1, crnt_enrl_intensity, term_gpa_1, has_term_gpa_1, tot_time_qrt1, overall_prop_comp, term_gpa_2, has_term_gpa_2',
# 20: 'avg_g, past_avg_grade, has_past_avg_grade, disc_post_cnt_qtr1, assign_sub_cnt_qtr1, has_assign_sub_cnt_qtr1, prop_comp_sd, disc_reply_cnt_qtr1, gpa_trend, avg_depth_post_qtr1',
# 30: 'avg_word_tot_qtr1, tot_click_cnt_qrt1c, has_concurrent_qtr1, irreg_session_len_qrt1, avg_g_concurrent, has_avg_g_concurrent, cum_cred_earn, tot_time_qrt1c, on_time_assign_share_qtr1, has_on_time_assign_share_qtr1',
# 40: 'avg_session_len_qrt1, irreg_session_len_qrt1c, avg_session_len_qrt1c, enrl_intensity_trend, section_size, enrl_intensity, age, prior_has_full, prior_tot_act_day_cnt, prior_tot_click_cnt',
# 50: 'has_assign_sub_cnt_qtr1c, assign_sub_cnt_qtr1c, pct_stopped, prior_tot_act_wk_cnt, num_of_prior_terms, prior_tot_click_cnt_qrt1, prior_has_qtr1, lvl2_share, prior_tot_time, prior_disc_post_cnt',
# 60: 'pct_dev, prior_tot_session_cnt, prior_disc_reply_cnt, prior_tot_time_qrt1, online_share, prior_avg_session_len, prior_irreg_session_len, prior_irreg_session_len_qrt1, prior_avg_session_len_qrt1, prior_avg_word_tot',
# 70: 'prior_assign_sub_cnt, prior_has_assign_sub_cnt, prior_avg_depth_post, prior_disc_post_cnt_qtr1, prior_on_time_assign_share, prior_has_on_time_assign_share, prior_disc_reply_cnt_qtr1, prior_avg_depth_post_qtr1, prior_avg_word_tot_qtr1, eve_share',
# 80: 'prior_assign_sub_cnt_qtr1, prior_has_assign_sub_cnt_qtr1, prior_on_time_assign_share_qtr1, prior_has_on_time_assign_share_qtr1, on_time_assign_share_qtr1c, has_on_time_assign_share_qtr1c, HUM_SOC_grade, HUM_SOC, SOC_SOC_grade, SOC_SOC',
# 90: 'HUM_MTH_grade, HUM_MTH, prereq_grade, has_prereq_grade, HUM_HUM_grade, HUM_HUM, summer_ind, pct_incomplete, HUM_SCI_grade, HUM_SCI',
# 100: 'SCI_MED_grade, SCI_MED, online_ind, SOC_MTH_grade, SOC_MTH, SOC_HUM_grade, SOC_HUM, full_time, MED_MED_grade, MED_MED',
# 110: 'SOC_SCI_grade, SOC_SCI, HUM_MED_grade, HUM_MED, MTH_SOC_grade, MTH_SOC, SCI_SCI_grade, SCI_SCI, degree_level_1, college_NVCC',
# 120: 'MTH_MTH_grade, MTH_MTH, tenure, MTH_SCI_grade, MTH_SCI, SCI_SOC_grade, SCI_SOC, cip_24, ever_dual, lvl2_ind',
# 130: 'MTH_HUM_grade, MTH_HUM, has_repeat_grade, repeat_grade, HUM_EGR_grade, HUM_EGR, SOC_MED_grade, SOC_MED, EGR_EGR_grade, EGR_EGR',
# 140: 'EGR_SOC_grade, EGR_SOC, college_TCC, degree_level_2, SCI_MTH_grade, SCI_MTH, SCI_HUM_grade, SCI_HUM, MTH_EGR_grade, MTH_EGR',
# 150: 'SOC_EGR_grade, SOC_EGR, EGR_MTH_grade, EGR_MTH, EGR_HUM_grade, EGR_HUM, MED_SCI_grade, MED_SCI, dev, cip_52',
# 160: 'EGR_SCI_grade, EGR_SCI, MTH_MED_grade, MTH_MED, MED_SOC_grade, MED_SOC, eve_ind, MED_HUM_grade, MED_HUM, cip_51',
# 170: 'cip_30, HUM_BUS_grade, HUM_BUS, cip_11, BUS_BUS_grade, BUS_BUS, cip_45, college_JSRCC, SOC_BUS_grade, SOC_BUS',
# 180: 'degree_level_3, SCI_EGR_grade, SCI_EGR, college_JTCC, HUM_ART_grade, HUM_ART, college_TNCC, MED_MTH_grade, MED_MTH, college_GCC',
# 190: 'EGR_MED_grade, EGR_MED, ART_SOC_grade, ART_SOC, ART_SCI_grade, ART_SCI, BUS_SOC_grade, BUS_SOC, MTH_BUS_grade, MTH_BUS',
# 200: 'BUS_HUM_grade, BUS_HUM, MED_EGR_grade, MED_EGR, ART_HUM_grade, ART_HUM, EGR_BUS_grade, EGR_BUS, college_LFCC, cip_14',
# 210: 'SOC_ART_grade, SOC_ART, ART_MTH_grade, ART_MTH, college_VWCC, BUS_MTH_grade, BUS_MTH, FLA_HUM_grade, FLA_HUM, college_PVCC',
# 220: 'BUS_SCI_grade, BUS_SCI, BUS_EGR_grade, BUS_EGR, ART_EGR_grade, ART_EGR, MTH_ART_grade, MTH_ART, cip_43, college_NRCC',
# 230: 'FLA_SCI_grade, FLA_SCI, OCC_OCC_grade, OCC_OCC, FLA_SOC_grade, FLA_SOC, SCI_BUS_grade, SCI_BUS, FLA_MTH_grade, FLA_MTH',
# 240: 'college_DCC, college_SWVCC, ART_ART_grade, ART_ART, cip_99, MED_BUS_grade, MED_BUS, cip_50, college_CVCC, college_PHCC',
# 250: 'HUM_OCC_grade, HUM_OCC, cip_15, college_MECC, EGR_ART_grade, EGR_ART, SCI_ART_grade, SCI_ART, college_SSVCC, college_RCC',
# 260: 'ART_MED_grade, ART_MED, FLA_EGR_grade, FLA_EGR, HUM_FLA_grade, HUM_FLA, OCC_HUM_grade, OCC_HUM, college_VHCC, college_WCC',
# 270: 'SOC_FLA_grade, SOC_FLA, cip_19, ART_BUS_grade, ART_BUS, MED_ART_grade, MED_ART, OCC_EGR_grade, OCC_EGR, cip_47',
# 280: 'BUS_MED_grade, BUS_MED, SOC_OCC_grade, SOC_OCC, MTH_FLA_grade, MTH_FLA, MTH_OCC_grade, MTH_OCC, EGR_OCC_grade, EGR_OCC',
# 290: 'OCC_SCI_grade, OCC_SCI, OCC_SOC_grade, OCC_SOC, college_PDCCC, BUS_ART_grade, BUS_ART, FLA_MED_grade, FLA_MED, college_DSLCC',
# 300: 'SCI_FLA_grade, SCI_FLA, OCC_MTH_grade, OCC_MTH, FLA_FLA_grade, FLA_FLA, FLA_BUS_grade, FLA_BUS, cip_42, college_ESCC',
# 309: 'EGR_FLA_grade, EGR_FLA, MED_FLA_grade, MED_FLA, cip_48, SCI_OCC_grade, SCI_OCC, FLA_ART_grade, FLA_ART',
# 319: 'MED_OCC_grade, MED_OCC, OCC_MED_grade, OCC_MED, ART_FLA_grade, ART_FLA, OCC_BUS_grade, OCC_BUS, OCC_ART_grade, OCC_ART',
# 329: 'BUS_FLA_grade, BUS_FLA, ART_OCC_grade, ART_OCC, BUS_OCC_grade, BUS_OCC, OCC_FLA_grade, OCC_FLA, FLA_OCC_grade, FLA_OCC'}

In [9]:
raw_predictors = {\
5: 'pct_withdrawn, cum_gpa, tot_click_cnt_qrt1, crnt_enrl_intensity, tot_time_qrt1',
15: 'term_gpa_1, has_term_gpa_1, overall_prop_comp, term_gpa_2, has_term_gpa_2, avg_g, past_avg_grade, has_past_avg_grade, disc_post_cnt_qtr1, prop_comp_sd',
25: 'assign_sub_cnt_qtr1, has_assign_sub_cnt_qtr1, disc_reply_cnt_qtr1, gpa_trend, avg_depth_post_qtr1, avg_word_tot_qtr1, has_concurrent_qtr1, tot_click_cnt_qrt1c, irreg_session_len_qrt1, cum_cred_earn',
35: 'avg_g_concurrent, has_avg_g_concurrent, tot_time_qrt1c, on_time_assign_share_qtr1, has_on_time_assign_share_qtr1, avg_session_len_qrt1, irreg_session_len_qrt1c, avg_session_len_qrt1c, enrl_intensity_trend, section_size',
45: 'enrl_intensity, age, prior_has_full, prior_tot_act_day_cnt, prior_tot_click_cnt, has_assign_sub_cnt_qtr1c, assign_sub_cnt_qtr1c, pct_stopped, prior_tot_act_wk_cnt, num_of_prior_terms'}

In [10]:
clean_predictors = {}
for k,v in raw_predictors.items():
    clean_predictors[k] = [p.strip() for p in v.split(",")]
num_of_predictors = sorted(list(clean_predictors.keys()))
clean_predictors_2 = {}
for k,v in clean_predictors.items():
    clean_predictors_2[k] = []
    for n in num_of_predictors:
        if n <= k:
            clean_predictors_2[k] += clean_predictors[n]
    assert len(clean_predictors_2[k]) == k

In [11]:
def create_cv_folds(train, predictors, n_fold = 5):
    folds = []
    k_fold = StratifiedKFold(n_splits = n_fold, random_state = 12345, shuffle=True)
    for train_indices, test_indices in k_fold.split(train, train.grade):
        train_part = train.iloc[train_indices,:]
        test_part = train.iloc[test_indices,:]
        X_1 = train_part.loc[:,predictors]
        y_1 = train_part.grade
        X_2 = test_part.loc[:,predictors]
        y_2 = test_part.grade
        folds.append([(X_1.copy(),y_1.copy()),(X_2.copy(),y_2.copy())])
    return folds

In [12]:
def cross_validation_RF(rf_model, folds):
    auc_by_fold = []
    for f in folds:
        X_1 = f[0][0]
        y_1 = f[0][1]
        X_2 = f[1][0]
        y_2 = f[1][1]
        rf_model.fit(X_1,y_1)
        y_2_pred = rf_model.predict_proba(X_2)[:,1]
        auc_by_fold.append(roc_auc_score(y_2,y_2_pred))
    return round(np.mean(auc_by_fold),4)  

In [13]:
def calc_cw(y):
    # Calculate the weight of each letter grade to be used in the modeling fitting procedure: the weight is inversely proportional to the square root of the frequency of the letter grade in the training sample
    cw = Counter(y)
    class_weight = {k:np.sqrt(cw.most_common()[0][-1]/v, dtype=np.float32) for k,v in cw.items()}
    return class_weight # The output is a dictionary mapping letter grade to the corresponding weight

In [14]:
cstat_by_num = []
for n in sorted(list(clean_predictors_2.keys())):
    print("n = {}:\n".format(n))
    predictors_list = clean_predictors_2[n]
    five_folds = create_cv_folds(train_df, predictors_list)
    auc_by_d=[]
    for d in range(5,36):
        rf = RandomForestClassifier(n_estimators=200, criterion="entropy", 
                                    max_depth=d,
                                    random_state=0, n_jobs=20, max_features="auto",
                                    class_weight = calc_cw(train_df.grade))
        auc = cross_validation_RF(rf, five_folds)
        auc_by_d.append(auc)
        print("max_depth = {0}: {1}".format(d, auc))
        if d > 5:
            if auc - auc_by_d[-2] < 0.001:
                break
            else:
                best_d = d
        else:
            best_d = d
    rf = RandomForestClassifier(n_estimators=200, criterion="entropy",
                                max_depth=best_d,
                                random_state=0, n_jobs=-1, max_features="auto",
                                class_weight = calc_cw(train_df.grade))
    rf.fit(train_df.loc[:,predictors_list], train_df.grade)
    cstat_by_num.append(round(roc_auc_score(test_df.grade, rf.predict_proba(test_df.loc[:,predictors_list])[:,1]),4))
    print("\nC-statistic = {}".format(cstat_by_num[-1]))
    print("\n\n\n")

n = 5:

max_depth = 5: 0.8096
max_depth = 6: 0.8121
max_depth = 7: 0.8139
max_depth = 8: 0.8152
max_depth = 9: 0.8164
max_depth = 10: 0.8174
max_depth = 11: 0.8184
max_depth = 12: 0.8192

C-statistic = 0.8338




n = 15:

max_depth = 5: 0.8253
max_depth = 6: 0.8304
max_depth = 7: 0.8338
max_depth = 8: 0.8367
max_depth = 9: 0.8391
max_depth = 10: 0.8411
max_depth = 11: 0.8428
max_depth = 12: 0.8443
max_depth = 13: 0.8456
max_depth = 14: 0.8468
max_depth = 15: 0.848
max_depth = 16: 0.849
max_depth = 17: 0.8501
max_depth = 18: 0.8509

C-statistic = 0.8582




n = 25:

max_depth = 5: 0.8292
max_depth = 6: 0.8341
max_depth = 7: 0.8385
max_depth = 8: 0.842
max_depth = 9: 0.845
max_depth = 10: 0.8479
max_depth = 11: 0.8504
max_depth = 12: 0.8525
max_depth = 13: 0.8543
max_depth = 14: 0.8559
max_depth = 15: 0.8573
max_depth = 16: 0.8586
max_depth = 17: 0.8597
max_depth = 18: 0.8605

C-statistic = 0.8674




n = 35:

max_depth = 5: 0.8292
max_depth = 6: 0.8341
max_depth = 7: 0.8387
max_depth = 

In [15]:
cstat_by_num

[0.8338, 0.8582, 0.8674, 0.8703, 0.8713]